[View in Colaboratory](https://colab.research.google.com/github/pranaynanda/LogFileChallenge/blob/master/Time_Series_Histogram_and_Percentile.ipynb)

# Set Project

In [0]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = '[your Cloud Platform project ID]'
!gcloud config set project bookshelf-1860044

# Get Files

This above part could be removed. I used Google Colaboratory which are basically Google hosted Jupyter notebooks. The code above authenticates my GCP console to the notebook. I could make the files publically available either to all users or all Google authenticated users which would eliminate the need to autheticate or set project. 


---


The code below could also be removed if we directly provide the file names. I did not want to copy them in the temporary instance over and over again.

In [0]:
# Download the file from a given Google Cloud Storage bucket.
!gsutil cp gs://ihopeitworks/* .
  
# Print the result to make sure the transfer worked.
#!cat /tmp/gsutil_download.txt

# Import Packages

In [0]:
import re
import pandas as pd
import numpy as np
from datetime import datetime

# Open File in Read mode

In [0]:
x = open('out.log', 'r')

# Define regex

I used regex the Splunk way. Splunk would run regex through each line of the input and then extract fields and values.

Not that Splunk does not work well with delimited data and although, our data is delimited to some extent, we run into the trouble of stripping braces and grouping separated words as error messages.

In [0]:
regex="^((\d{4}-\d{2}-\d{2})\s(\d{2}:\d{2}:\d{2}\.\d+))\s\[PID (\d+)\]\s\[(\d+)\w{2}\]\s\[UID (\w+)\]\s\[(\w+)\]\s(\/\w+)(.*Error name: (\w.+))?(.*)$"

# Run Regex

`re.findall()` gives us a list of tuples if the strings match. It also gives us match groups. The `regex_read` value stores a list of tuples. We extract values from the tuples and create a list of values of a particular field type.

In [0]:
regex_read = re.findall(regex, x.read(), re.MULTILINE)

# The regex_read value is a list of tuples. Although, I could have created a list of lists here or even a numpy map, 
# I chose to extract them seperately for readability, 

timestamps = [datetime.strptime(match[0], "%Y-%m-%d %H:%M:%S.%f") for match in regex_read] # creating lists from the first tuple from the list 'regex_read' and then converting it into datetime values
date = [match[1] for match in regex_read]
time = [match[2] for match in regex_read]
pid = [match[3] for match in regex_read]
response_time_in_ms = [int(match[4]) for match in regex_read]
uid = [match[5] for match in regex_read]
log_level = [match[6] for match in regex_read]
url = [match[7] for match in regex_read]
error_message = [match[8] for match in regex_read]
error_type = [match[9] for match in regex_read]
message = [match[10] for match in regex_read]



# Create Dataframe

In [0]:
#Create DataFrame called logfile with series names that pull values from lists that were extracted above

logfile=pd.DataFrame({'Timestamp': timestamps,
                       'PID': pid,
                       'Response Time': response_time_in_ms,
                       'UID': uid,
                       'Log Level': log_level,
                       'URL': url,
                       'Error Message': error_message,
                       'Error Type': error_type,
                       'Message': message})

#The regex isn't as neat as I would like it to be. Therefore there are values in list that were captured as empty from the log data 
# We replace the empty values with np.nan value from numpy

logfile=logfile.replace('', np.nan)


#We check datatypes to be sure that they are as needed.

logfile.dtypes

# Extract Histogram for the Given dates
Because the data is such that the last row has the earliest timestamp, we start the series from bottom and create a date_time index with the values that have an interval frequency of seconds.

In my experience with the monitoring tools as an SRE, there often isn't enough information at the microsecond scale. Seconds does the job.

In [0]:
#Converting values to datetime again for redundancy
logfile['Timestamp'] = pd.to_datetime(logfile['Timestamp'])

# We could also create a function here

start_time='2018-08-30 16:02:25.846066'
end_time='2018-08-30 17:12:28.846066'

#Make a date_range with frequency in seconds. 
logfile['Timestamp'] = pd.date_range(start=logfile['Timestamp'].iloc[-1], end=logfile['Timestamp'].iloc[0],freq='S')

#Create a new logfile dataframe that contains the values where the timestamp is between specified range.
logfile=logfile[(logfile['Timestamp'] > start_time) & (logfile['Timestamp'] <= end_time)] 

#Output histogram
logfile['Error Message'].hist()


# Extract Percentile of error messages for given dates

The new data frame created above is also used here. We did not capture the log trace messages. Therefore we drop the messages that contain the values `np.nan` which would then only show values for successful responses. Then pass it to the `.describe()` function for percentiles. 

Because response time is the only integer value in the entire dataframe, we only get percentiles for response time. 

*PID and UID may be numbers but they are contextual and not arithemetic. *



---



We could also use the `.quantile()` function. For simplicity, I used `.describe()`. 

In [0]:
logfile.dropna(subset=['Message']).describe(percentiles=[0.50,0.90,0.95])